In [1]:
####http://stackoverflow.com/questions/8376691/how-to-remove-hashtag-user-link-of-a-tweet-using-regular-expression
import re,string

def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def strip_all_entities(text):
    ###strip only @mentions and not #hashtags!
    entity_prefixes = ['@']
#     entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

# tests = [
#     "@peter I really love that shirt at #Macy. http://bet.ly//WjdiW4",
#     "@shawn Titanic tragedy could have been prevented Economic Times: Telegraph.co.ukTitanic tragedy could have been preve... http://bet.ly/tuN2wx",
#     "I am at Starbucks http://4sh.com/samqUI (7419 3rd ave, at 75th, Brooklyn)",
# ]
# for t in tests:
#     print strip_all_entities(strip_links(t))

In [2]:
#remove @mentions and URLs in tweets - preprocessing
filename = ['pos_examples_happy.txt', 'neg_examples_sad.txt','pos_examples_PosSentiment.txt', 'neg_examples_NegSentiment.txt']
for file_ in filename:
    print file_
    target = open('data/nohashtags'+file_, 'w')
    count = 0

    with open('data/'+file_, 'r') as f:
        pos_tweets = f.readlines()
        for tweet in pos_tweets:
            count += 1
            target.write(strip_all_entities(strip_links(tweet)))
            target.write('\n')

print "Done! with pre-processing"

Done! with pre-processing


In [3]:
###test on preprocessed data

##### load training data
import random
import numpy as np
import pandas as pd

# suffix = ""
suffix = "preprocessed_"
###read tagged sentiment Tweets
with open('data/'+suffix+'pos_examples_PosSentiment.txt', 'r') as f:
    pos_tweets = f.readlines()

with open('data/'+suffix+'neg_examples_NegSentiment.txt', 'r') as f:
    neg_tweets = f.readlines()
    
class_label = 0
train_pos = pd.DataFrame(pos_tweets, columns = ['text'])
X0 = train_pos.text
y0 = np.full(shape=len(X0), fill_value=class_label)

###create random 80-20 split
rows = random.sample(X0.index, int(0.8*len(X0)))
X0_80 = X0.ix[rows]
X0_20 = X0.drop(rows)
y0_80 = y0[:int(0.8*len(X0))]
y0_20 = y0[int(0.8*len(X0)):]
###

class_label = 1
train_neg = pd.DataFrame(neg_tweets, columns = ['text'])
X1 = train_neg.text
y1 = np.full(shape=len(X1), fill_value=class_label)

###create random 80-20 split
rows = random.sample(X1.index, int(0.8*len(X1)))
X1_80 = X1.ix[rows]
X1_20 = X1.drop(rows)
y1_80 = y1[:int(0.8*len(X1))]
y1_20 = y1[int(0.8*len(X1)):]
###

X_train = pd.concat((X0_80,X1_80))
y_train = np.concatenate((y0_80, y1_80))

X_holdout = pd.concat((X0_20,X1_20))
y_holdout = np.concatenate((y0_20, y1_20))

### load testing data

###read tagged sentiment Tweets
with open('data/'+suffix+'pos_examples_happy.txt', 'r') as f:
    pos_tweets = f.readlines()

with open('data/'+suffix+'neg_examples_sad.txt', 'r') as f:
    neg_tweets = f.readlines()
    
import numpy as np
class_label = 0
train_pos = pd.DataFrame(pos_tweets, columns = ['text'])
X0 = train_pos.text
y0 = np.full(shape=len(X0), fill_value=class_label)

class_label = 1
train_neg = pd.DataFrame(neg_tweets, columns = ['text'])
X1 = train_neg.text
y1 = np.full(shape=len(X1), fill_value=class_label)

X_test = pd.concat((X0,X1))
y_test = np.concatenate((y0, y1))

###
print "Loading data = COMPLETE!"

Loading data = COMPLETE!


In [4]:
##Generate top 5000 ngrams and select random 50
###print top 5000 features 
import nltk
from sklearn.feature_extraction.text import CountVectorizer

##vectorizer arguments blah!

tokenizer=None#word_tokenize
stop_words=nltk.corpus.stopwords.words("english")#None
stop_words.append('rt')
ngram_range=(1, 3)
lowercase=True
max_features=5000
binary=False
dtype=np.float64

###create vectorizer
vectorizer = CountVectorizer(decode_error="ignore",
#                                tokenizer=tokenizer,
                               stop_words=stop_words,
                               ngram_range=ngram_range,
                               lowercase=lowercase,
                               binary=binary,
                               dtype=dtype,
                               max_features=max_features)

train_fit = vectorizer.fit_transform(X_train)

vocab = vectorizer.vocabulary_
arr = (train_fit.toarray().sum(axis=0))

with open("data/top5000.txt", "w") as f:
    for key in vocab:
        try:
            f.write(str(key) + ',' + str(arr[vocab[key]]) + "\n")
        except:
            continue
            
print "Done getting top 5000 ngrams!"

Done getting top 5000 ngrams!


In [ ]:
### 5-fold cross validation 
###build pipeline; fit on BEST CV MODEL train; predict on test!
import nltk
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold

###5-fold cross validation
n_folds = 5
kf = KFold(len(X_train), n_folds=n_folds, shuffle=True, random_state=42)

In [14]:
###create pipeline
classifier = "LinearSVC"
text_clf = Pipeline([
                     ('vect', vectorizer),
#                      ('clf', MultinomialNB(alpha=0.1)),
                    ('clf', LinearSVC(C=1.0)),
#         ("clf", RandomForestClassifier(n_estimators=100, max_features="auto", criterion="entropy", n_jobs=-1)),
                    ])


count = 0
max_fscore = 0.0
best_X_train = X_train
best_y_train = y_train
best_model = 0

for train, test in kf:
    count = count + 1
    print "Fold ", count
    X_CVtrain, y_CVtrain, X_CVtest, y_CVtest = X_train.values[train], y_train[train], X_train.values[test], y_train[test]
    ###fit training data
    text_clf = text_clf.fit(X_CVtrain, y_CVtrain)
    
    ### predict on test data
    predicted = text_clf.predict(X_CVtest)

    ###get performance metrics
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support
    accuracy = accuracy_score(y_CVtest,predicted)
    precision, recall, fscore, sup = precision_recall_fscore_support(y_CVtest, predicted, average='binary', pos_label=0)
    print classifier, accuracy, precision, recall, fscore
    
    ###update best-model
    if max_fscore < fscore: 
        max_fscore = fscore
        best_X_train = X_CVtrain
        best_y_train = y_CVtrain
        best_model = count - 1
        
###fit the best SVC model
text_clf.fit(best_X_train, best_y_train)

print "predict on holdout data"
predicted = text_clf.predict(X_holdout)

###get performance metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(y_holdout,predicted)
precision, recall, fscore, sup = precision_recall_fscore_support(y_holdout, predicted, average='binary', pos_label=0)
print accuracy, precision, recall, fscore

print "predict on test data"
predicted = text_clf.predict(X_test)

###get performance metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(y_test,predicted)
precision, recall, fscore, sup = precision_recall_fscore_support(y_test, predicted, average='binary', pos_label=0)
print accuracy, precision, recall, fscore

Fold  1
LinearSVC 0.73578125 0.708153302284 0.795010709336 0.749072562695
Fold  2
LinearSVC 0.73340625 0.713484091802 0.787794299876 0.748800094226
Fold  3
LinearSVC 0.73640625 0.711582048957 0.789662327863 0.748591696224
Fold  4
LinearSVC 0.73459375 0.71769871137 0.780395560791 0.747735170939
Fold  5
LinearSVC 0.735125 0.711625537695 0.788078224897 0.747903158646
predict on holdout data
0.738425 0.714594302687 0.79395 0.752184931669
predict on test data
0.960935738444 0.959020994723 0.963021420519 0.961017044496


In [15]:
###create pipeline
classifier = "MNB"
text_clf = Pipeline([
                     ('vect', vectorizer),
                     ('clf', MultinomialNB(alpha=0.1)),
#                     ('clf', LinearSVC(C=1.0)),
#         ("clf", RandomForestClassifier(n_estimators=100, max_features="auto", criterion="entropy", n_jobs=-1)),
                    ])


count = 0
max_fscore = 0.0
best_X_train = X_train
best_y_train = y_train
best_model = 0

for train, test in kf:
    count = count + 1
    print "Fold ", count
    X_CVtrain, y_CVtrain, X_CVtest, y_CVtest = X_train.values[train], y_train[train], X_train.values[test], y_train[test]
    ###fit training data
    text_clf = text_clf.fit(X_CVtrain, y_CVtrain)
    ### predict on test data
    predicted = text_clf.predict(X_CVtest)

    ###get performance metrics
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support
    accuracy = accuracy_score(y_CVtest,predicted)
    precision, recall, fscore, sup = precision_recall_fscore_support(y_CVtest, predicted, average='binary', pos_label=0)
    print classifier, accuracy, precision, recall, fscore
    
    ###update best-model
    if max_fscore < fscore: 
        max_fscore = fscore
        best_X_train = X_CVtrain
        best_y_train = y_CVtrain
        best_model = count - 1
        
###fit the best SVC model
text_clf.fit(best_X_train, best_y_train)

print "predict on holdout data"
predicted = text_clf.predict(X_holdout)

###get performance metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(y_holdout,predicted)
precision, recall, fscore, sup = precision_recall_fscore_support(y_holdout, predicted, average='binary', pos_label=0)
print accuracy, precision, recall, fscore

print "predict on test data"
predicted = text_clf.predict(X_test)

###get performance metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(y_test,predicted)
precision, recall, fscore, sup = precision_recall_fscore_support(y_test, predicted, average='binary', pos_label=0)
print accuracy, precision, recall, fscore

Fold  1
MNB 0.7289375 0.722249660452 0.736991306539 0.729546021452
Fold  2
MNB 0.72653125 0.73982473223 0.706133828996 0.722586780789
Fold  3
MNB 0.73078125 0.724494825037 0.739483116393 0.731912245215
Fold  4
MNB 0.729 0.744379629023 0.704135408271 0.723698464283
Fold  5
MNB 0.7294375 0.728057014254 0.729973674314 0.729014084507
predict on holdout data
0.732225 0.729414670289 0.73835 0.733855137284
predict on test data
0.955326944758 0.944380260769 0.967643742954 0.95587047916


In [16]:
###create pipeline
classifier = "RF"
text_clf = Pipeline([
                     ('vect', vectorizer),
#                      ('clf', MultinomialNB(alpha=0.1)),
#                     ('clf', LinearSVC(C=1.0)),
        ("clf", RandomForestClassifier(n_estimators=100, max_features="auto", criterion="entropy", n_jobs=-1)),
                    ])


count = 0
max_fscore = 0.0
best_X_train = X_train
best_y_train = y_train
best_model = 0

for train, test in kf:
    count = count + 1
    print "Fold ", count
    X_CVtrain, y_CVtrain, X_CVtest, y_CVtest = X_train.values[train], y_train[train], X_train.values[test], y_train[test]
    ###fit training data
    text_clf = text_clf.fit(X_CVtrain, y_CVtrain)
    ### predict on test data
    predicted = text_clf.predict(X_CVtest)

    ###get performance metrics
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support
    accuracy = accuracy_score(y_CVtest,predicted)
    precision, recall, fscore, sup = precision_recall_fscore_support(y_CVtest, predicted, average='binary', pos_label=0)
    print classifier, accuracy, precision, recall, fscore
    
    ###update best-model
    if max_fscore < fscore: 
        max_fscore = fscore
        best_X_train = X_CVtrain
        best_y_train = y_CVtrain
        best_model = count - 1
        
###fit the best SVC model
text_clf.fit(best_X_train, best_y_train)

print "predict on holdout data"
predicted = text_clf.predict(X_holdout)

###get performance metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(y_holdout,predicted)
precision, recall, fscore, sup = precision_recall_fscore_support(y_holdout, predicted, average='binary', pos_label=0)
print accuracy, precision, recall, fscore

print "predict on test data"
predicted = text_clf.predict(X_test)

###get performance metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(y_test,predicted)
precision, recall, fscore, sup = precision_recall_fscore_support(y_test, predicted, average='binary', pos_label=0)
print accuracy, precision, recall, fscore

Fold  1
RF 0.73115625 0.718309013391 0.753559279325 0.735512036155
Fold  2
RF 0.73484375 0.726011219368 0.761771995043 0.743461829176
Fold  3
RF 0.7293125 0.715801394767 0.755140539521 0.734944920441
Fold  4
RF 0.7309375 0.725349157825 0.750263500527 0.737596001463
Fold  5
RF 0.72921875 0.720918954962 0.745455685095 0.732982034452
predict on holdout data
0.731675 0.722989556764 0.75115 0.736800804336
predict on test data
0.97178692221 0.982753648267 0.960428410372 0.97146278188
